In [3]:
import requests

In [ ]:
class SpaceTrackAPI():
    """ Provides an API for making POST requests to space-track.org """
    def __init__(self, username: str, password: str, fmt: str=None):
        """ Initializes the API.
        
        This API class can be used to build and submit requests to space-track.org.
        
        Args:
            username: your space-track.org username.
            password: the associated password.
        
        Kwargs:
            fmt: string specifying format for returned message. Can be one of
            'xml', 'json', 'html', 'csv', 'tle', '3le', 'kvn', or None. None
            is the same as 'json'. Default is None.
        
        Raises:
            ValueError: if provided fmt is not one of the specified options.
        """
        valid_fmt = ['xml', 'json', 'html', 'csv', 'tle', '3le', 'kvn']
        if fmt is not None:
            if fmt.lower() not in valid_fmt:
                raise ValueError("fmt must be one of 'xml', 'json', 'html', \
                                 'csv', 'tle', '3le', 'kvn', or None.")
        else:
            fmt = 'json'
        self.fmt = fmt.lower()
        self.username = username
        self.password = password
        self.base = 'https://space-track.org/'  # base URL for requests
        self.login_url = 'https://www.space-track.org/ajaxauth/login'  # login URL
        self.logout_url = 'https://www.space-track.org/ajaxauth/logout'
        self.query = []  # placeholder for our query string
    
    def _logout(self) -> requests.models.Response:
        """ Logs out of the space-track.org session.
        
        Returns:
            Response from space-track.org
        """
        res = requests.post(self.logout_url)
        if not res.ok:
            print('Error logging out! Status code {}'.format(res.status_code))
        return res
    
    def _basic(self):
        """ Adds 'basicspacedata' to the query. """
        self.query.append('basicspacedata')
    
    def _query(self):
        """ Builds the start of a query. """
        self._basic()
        self.query.append('query')
        
    def _compile_query(self) -> str:
        """ Compiles the query (list of strings) into a '/'-separated string.
        
        Returns:
            The query string with each element separated by a forward-slash.
        """
        if self.fmt not in self.query:
            self.query.extend(['format', self.fmt])
        return '/'.join(self.query)
    
    def print_query(self):
        """ Prints the query to the screen.
        
        Note that at this stage the format string has not been applied to
        the query. This string gets applied at the very end prior to
        submitting the POST request.
        """
        print('/'.join(self.query))

    def tle(self):
        """ Initiates a tle request. """
        self.keys = ['NORAD_CAT_ID', 'EPOCH', 'MEAN_MOTION', 'ECCENTRICITY',
                     'INCLINATION', 'MEAN_ANOMALY', 'PERIOD']
        self._query()
        self.query.extend('class', 'tle')
    
    def tle_latest(self):
        """ Initiates a tle_latest request. """
        self._query()
        self.query.extend('class', 'tle_latest')
    
    def tle_publish(self):
        """ Initiates a tle_publish request. """
        self._query()
        self.query.extend('class', 'tle_publish')
    
    def boxscore(self):
        """ Initiates a boxscore request. """
        self._query()
        self.query.extend('class', 'boxscore')
        
    def satcat(self):
        """ Initiates a satcat request. """
        self._query()
        self.query.extend('class', 'satcat')
        
    def launch_site(self):
        """ Initiates a launch_site request. """
        self._query()
        self.query.extend('class', 'launch_site')
        
    def satcat_change(self):
        """ Initiates a satcat_change request. """
        self._query()
        self.query.extend('class', 'satcat_change')
        
    def satcat_debut(self):
        """ Initiates a satcat_debut request. """
        self._query()
        self.query.extend('class', 'satcat_debut')
        
    def decay(self):
        """ Initiates a decay request. """
        self._query()
        self.query.extend('class', 'decay')
        
    def announcement(self):
        """ Initiates a announcement request. """
        self._query()
        self.query.extend('class', 'announcement')
        
    def cdm(self):
        """ Initiates a cdm request. """
        self._query()
        self.query.extend('class', 'cdm')
        
    def organization(self):
        """ Initiates a organization request. """
        self._query()
        self.query.extend('class', 'organization')
    
    def submit(self) -> requests.models.Response:
        """ Submits the generated query to space-track.org and returns the response.
        
        Returns:
            Response from space-track.org
        """
        payload = {'identity': self.username,
                   'password': self.password,
                   'query': self._compile_query()}
        res = requests.post(self.login_url, data=payload)
        if not res.ok:
            print('Error posting request! Status code {}'.format(res.status_code))
        self._logout()
        return res